# Examine multi-processing / paralell execution option
    * result - may not be worth the effort because file writes may clash & hang
    
****
## scipy image: cleaning, segmentation, etc.
[API Reference: scikit-image](https://scikit-image.org/docs/dev/api/api.html) <br>
****
[Home: scikit-image aka skimage](https://scikit-image.org/) <br>
[Documentation: scikit-image](https://scikit-image.org/docs/stable/) <br>
[scikit-image on GitHub](https://github.com/scikit-image/scikit-image) <br>
[User Guide: scikit-image](https://scikit-image.org/docs/dev/user_guide) <br>
[Founder-tutor](https://bids.berkeley.edu/resources/videos/scikit-image-image-analysis-python-intermediate) <br>
****
## OpenSlide:
[OpenSlide Wiki](https://github.com/openslide/openslide/wiki) <br>
[OpenSlide api](https://openslide.org/api/python/) <br>
[openslide github](https://github.com/openslide/openslide-python) <br>
[openslide dot org](https://openslide.org/) <br>
[openslide C docs](https://openslide.org/api/openslide_8h.html) <br>

In [1]:
# import tempfile

# dirpath = tempfile.mkdtemp()
# if os.path.isdir(dirpath):
#     print(dirpath)
#     os.rmdir(dirpath)
#     print('directory still exists = ',os.path.isdir(dirpath))

In [2]:
import os
import time
import glob
import math
import multiprocessing as mp
import itertools
import hashlib
import tempfile

import numpy as np
import openslide
from skimage.filters import threshold_otsu

import PIL

output_dir = '../../DigiPath_MLTK_data/out_to_test'
if os.path.isdir(output_dir) == False:
    os.makedirs(output_dir)
    
OUTPUT_PATH = output_dir
data_dir = '../../DigiPath_MLTK_data/Aperio'
files_list = ['CMU-1-Small-Region.svs', 'CMU-1.svs']
SVS = os.path.join(data_dir, files_list[0])
print('Working with file:', SVS, '\nOutput path:', OUTPUT_PATH)

Working with file: ../../DigiPath_MLTK_data/Aperio/CMU-1-Small-Region.svs 
Output path: ../../DigiPath_MLTK_data/out_to_test


In [3]:
out_dir = '../../DigiPath_MLTK_data/out_to_test'
file_types = ['.jpg']
files_list = []
for maybe_file in os.listdir(out_dir):
    full_path = os.path.join(out_dir, maybe_file)
    if os.path.isfile(full_path):
        _, f_ext = os.path.splitext(maybe_file)
        if f_ext in file_types:
            files_list.append(full_path)

print(len(files_list))

999


In [4]:
"""                                                Serial time test
need to know what size the pre-trained models used
& graphic file types preferred by tensorflow (float, float32, int, etc)
"""
PATCH_HEIGHT = 256
PATCH_WIDTH = 256

t0 = time.time()
# open the file
test_file_name = os.path.join(data_dir, 'CMU-1.svs')
print('Opening Image file: \n\t{}\n'.format(test_file_name))
os_obj = openslide.OpenSlide(test_file_name)
pixels_height = os_obj.dimensions[0]
pixels_width = os_obj.dimensions[1]

print('Image is {} x {}\n{} pixels'.format(pixels_height, 
                                           pixels_width, 
                                           pixels_height * pixels_width))

# read the meta data
pixels_height_ds = os_obj.level_dimensions[-1][0]
pixels_width_ds = os_obj.level_dimensions[-1][1]
print(pixels_height_ds, pixels_width_ds)

# find the thumbnail size ratio (need pre-trained size)
print(pixels_height / pixels_height_ds, pixels_width / pixels_width_ds)
print('obj.level_downsamples', os_obj.level_downsamples)
print('obj.level_downsamples[-1]', os_obj.level_downsamples[-1])

big_thumb = os_obj.get_thumbnail((pixels_height_ds, pixels_width_ds))
print(type(big_thumb), big_thumb.size, big_thumb.getextrema())

# make the mask
grey_thumbnail = np.array(big_thumb.convert('L'))
thresh = threshold_otsu(grey_thumbnail)
mask = np.array(grey_thumbnail) < thresh
mask_im = PIL.Image.fromarray(np.uint8(mask) * 255)
# display(mask_im)

# make the grid
grid_height = pixels_height // PATCH_HEIGHT
grid_width = pixels_width // PATCH_WIDTH
print('\n%40s: %5i, %5i'%('grid size', grid_height, grid_width))
print('%40s: %5i, %5i'%('full size cells',PATCH_HEIGHT, PATCH_WIDTH))
thumb_cell_height = pixels_height_ds // grid_height
thumb_cell_width = pixels_width_ds // grid_width
print('%40s: %5i, %5i'%('thumb cells',thumb_cell_height, thumb_cell_width))

# make the directory
_, f_name = os.path.split(test_file_name)
f_name, f_ext = os.path.splitext(f_name)
print('directory name',f_name)

# get the ndarrays of full size grid, write as type
zoom_up_ratio = 16
for grid_row in range(grid_height):
    for grid_col in range(grid_width):
        h_idx_start = grid_row * thumb_cell_height
        h_idx_end = (grid_row + 1) * thumb_cell_height
        w_idx_start = grid_col * thumb_cell_width
        w_idx_end = (grid_col + 1) * thumb_cell_width
        
        big_idx = np.array([h_idx_start,h_idx_end,w_idx_start,w_idx_end])*zoom_up_ratio
        print(h_idx_start, h_idx_end, w_idx_start, w_idx_end, '\n\t', big_idx)
        
print('\nthumb size:', pixels_height_ds, pixels_width_ds)
print('\nfull size:', pixels_height, pixels_width)

# (save a list of files)

print('\ncell run time', time.time() - t0)

Opening Image file: 
	../../DigiPath_MLTK_data/Aperio/CMU-1.svs

Image is 46000 x 32914
1514044000 pixels
2875 2057
16.0 16.000972289742343
obj.level_downsamples (1.0, 4.000121536217793, 16.00048614487117)
obj.level_downsamples[-1] 16.00048614487117
<class 'PIL.Image.Image'> (2875, 2057) ((0, 255), (0, 255), (3, 255))

                               grid size:   179,   128
                         full size cells:   256,   256
                             thumb cells:    16,    16
directory name CMU-1
0 16 0 16 
	 [  0 256   0 256]
0 16 16 32 
	 [  0 256 256 512]
0 16 32 48 
	 [  0 256 512 768]
0 16 48 64 
	 [   0  256  768 1024]
0 16 64 80 
	 [   0  256 1024 1280]
0 16 80 96 
	 [   0  256 1280 1536]
0 16 96 112 
	 [   0  256 1536 1792]
0 16 112 128 
	 [   0  256 1792 2048]
0 16 128 144 
	 [   0  256 2048 2304]
0 16 144 160 
	 [   0  256 2304 2560]
0 16 160 176 
	 [   0  256 2560 2816]
0 16 176 192 
	 [   0  256 2816 3072]
0 16 192 208 
	 [   0  256 3072 3328]
0 16 208 224 
	 [   0  25

32 48 1088 1104 
	 [  512   768 17408 17664]
32 48 1104 1120 
	 [  512   768 17664 17920]
32 48 1120 1136 
	 [  512   768 17920 18176]
32 48 1136 1152 
	 [  512   768 18176 18432]
32 48 1152 1168 
	 [  512   768 18432 18688]
32 48 1168 1184 
	 [  512   768 18688 18944]
32 48 1184 1200 
	 [  512   768 18944 19200]
32 48 1200 1216 
	 [  512   768 19200 19456]
32 48 1216 1232 
	 [  512   768 19456 19712]
32 48 1232 1248 
	 [  512   768 19712 19968]
32 48 1248 1264 
	 [  512   768 19968 20224]
32 48 1264 1280 
	 [  512   768 20224 20480]
32 48 1280 1296 
	 [  512   768 20480 20736]
32 48 1296 1312 
	 [  512   768 20736 20992]
32 48 1312 1328 
	 [  512   768 20992 21248]
32 48 1328 1344 
	 [  512   768 21248 21504]
32 48 1344 1360 
	 [  512   768 21504 21760]
32 48 1360 1376 
	 [  512   768 21760 22016]
32 48 1376 1392 
	 [  512   768 22016 22272]
32 48 1392 1408 
	 [  512   768 22272 22528]
32 48 1408 1424 
	 [  512   768 22528 22784]
32 48 1424 1440 
	 [  512   768 22784 23040]
32 48 1440

80 96 272 288 
	 [1280 1536 4352 4608]
80 96 288 304 
	 [1280 1536 4608 4864]
80 96 304 320 
	 [1280 1536 4864 5120]
80 96 320 336 
	 [1280 1536 5120 5376]
80 96 336 352 
	 [1280 1536 5376 5632]
80 96 352 368 
	 [1280 1536 5632 5888]
80 96 368 384 
	 [1280 1536 5888 6144]
80 96 384 400 
	 [1280 1536 6144 6400]
80 96 400 416 
	 [1280 1536 6400 6656]
80 96 416 432 
	 [1280 1536 6656 6912]
80 96 432 448 
	 [1280 1536 6912 7168]
80 96 448 464 
	 [1280 1536 7168 7424]
80 96 464 480 
	 [1280 1536 7424 7680]
80 96 480 496 
	 [1280 1536 7680 7936]
80 96 496 512 
	 [1280 1536 7936 8192]
80 96 512 528 
	 [1280 1536 8192 8448]
80 96 528 544 
	 [1280 1536 8448 8704]
80 96 544 560 
	 [1280 1536 8704 8960]
80 96 560 576 
	 [1280 1536 8960 9216]
80 96 576 592 
	 [1280 1536 9216 9472]
80 96 592 608 
	 [1280 1536 9472 9728]
80 96 608 624 
	 [1280 1536 9728 9984]
80 96 624 640 
	 [ 1280  1536  9984 10240]
80 96 640 656 
	 [ 1280  1536 10240 10496]
80 96 656 672 
	 [ 1280  1536 10496 10752]
80 96 672 688

	 [ 2048  2304 12544 12800]
128 144 800 816 
	 [ 2048  2304 12800 13056]
128 144 816 832 
	 [ 2048  2304 13056 13312]
128 144 832 848 
	 [ 2048  2304 13312 13568]
128 144 848 864 
	 [ 2048  2304 13568 13824]
128 144 864 880 
	 [ 2048  2304 13824 14080]
128 144 880 896 
	 [ 2048  2304 14080 14336]
128 144 896 912 
	 [ 2048  2304 14336 14592]
128 144 912 928 
	 [ 2048  2304 14592 14848]
128 144 928 944 
	 [ 2048  2304 14848 15104]
128 144 944 960 
	 [ 2048  2304 15104 15360]
128 144 960 976 
	 [ 2048  2304 15360 15616]
128 144 976 992 
	 [ 2048  2304 15616 15872]
128 144 992 1008 
	 [ 2048  2304 15872 16128]
128 144 1008 1024 
	 [ 2048  2304 16128 16384]
128 144 1024 1040 
	 [ 2048  2304 16384 16640]
128 144 1040 1056 
	 [ 2048  2304 16640 16896]
128 144 1056 1072 
	 [ 2048  2304 16896 17152]
128 144 1072 1088 
	 [ 2048  2304 17152 17408]
128 144 1088 1104 
	 [ 2048  2304 17408 17664]
128 144 1104 1120 
	 [ 2048  2304 17664 17920]
128 144 1120 1136 
	 [ 2048  2304 17920 18176]
128 144 11

	 [2816 3072 5376 5632]
176 192 352 368 
	 [2816 3072 5632 5888]
176 192 368 384 
	 [2816 3072 5888 6144]
176 192 384 400 
	 [2816 3072 6144 6400]
176 192 400 416 
	 [2816 3072 6400 6656]
176 192 416 432 
	 [2816 3072 6656 6912]
176 192 432 448 
	 [2816 3072 6912 7168]
176 192 448 464 
	 [2816 3072 7168 7424]
176 192 464 480 
	 [2816 3072 7424 7680]
176 192 480 496 
	 [2816 3072 7680 7936]
176 192 496 512 
	 [2816 3072 7936 8192]
176 192 512 528 
	 [2816 3072 8192 8448]
176 192 528 544 
	 [2816 3072 8448 8704]
176 192 544 560 
	 [2816 3072 8704 8960]
176 192 560 576 
	 [2816 3072 8960 9216]
176 192 576 592 
	 [2816 3072 9216 9472]
176 192 592 608 
	 [2816 3072 9472 9728]
176 192 608 624 
	 [2816 3072 9728 9984]
176 192 624 640 
	 [ 2816  3072  9984 10240]
176 192 640 656 
	 [ 2816  3072 10240 10496]
176 192 656 672 
	 [ 2816  3072 10496 10752]
176 192 672 688 
	 [ 2816  3072 10752 11008]
176 192 688 704 
	 [ 2816  3072 11008 11264]
176 192 704 720 
	 [ 2816  3072 11264 11520]
176 192 7

...

2608 2624 624 640 
	 [41728 41984  9984 10240]
2608 2624 640 656 
	 [41728 41984 10240 10496]
2608 2624 656 672 
	 [41728 41984 10496 10752]
2608 2624 672 688 
	 [41728 41984 10752 11008]
2608 2624 688 704 
	 [41728 41984 11008 11264]
2608 2624 704 720 
	 [41728 41984 11264 11520]
2608 2624 720 736 
	 [41728 41984 11520 11776]
2608 2624 736 752 
	 [41728 41984 11776 12032]
2608 2624 752 768 
	 [41728 41984 12032 12288]
2608 2624 768 784 
	 [41728 41984 12288 12544]
2608 2624 784 800 
	 [41728 41984 12544 12800]
2608 2624 800 816 
	 [41728 41984 12800 13056]
2608 2624 816 832 
	 [41728 41984 13056 13312]
2608 2624 832 848 
	 [41728 41984 13312 13568]
2608 2624 848 864 
	 [41728 41984 13568 13824]
2608 2624 864 880 
	 [41728 41984 13824 14080]
2608 2624 880 896 
	 [41728 41984 14080 14336]
2608 2624 896 912 
	 [41728 41984 14336 14592]
2608 2624 912 928 
	 [41728 41984 14592 14848]
2608 2624 928 944 
	 [41728 41984 14848 15104]
2608 2624 944 960 
	 [41728 41984 15104 15360]
2608 2624 960

2656 2672 1152 1168 
	 [42496 42752 18432 18688]
2656 2672 1168 1184 
	 [42496 42752 18688 18944]
2656 2672 1184 1200 
	 [42496 42752 18944 19200]
2656 2672 1200 1216 
	 [42496 42752 19200 19456]
2656 2672 1216 1232 
	 [42496 42752 19456 19712]
2656 2672 1232 1248 
	 [42496 42752 19712 19968]
2656 2672 1248 1264 
	 [42496 42752 19968 20224]
2656 2672 1264 1280 
	 [42496 42752 20224 20480]
2656 2672 1280 1296 
	 [42496 42752 20480 20736]
2656 2672 1296 1312 
	 [42496 42752 20736 20992]
2656 2672 1312 1328 
	 [42496 42752 20992 21248]
2656 2672 1328 1344 
	 [42496 42752 21248 21504]
2656 2672 1344 1360 
	 [42496 42752 21504 21760]
2656 2672 1360 1376 
	 [42496 42752 21760 22016]
2656 2672 1376 1392 
	 [42496 42752 22016 22272]
2656 2672 1392 1408 
	 [42496 42752 22272 22528]
2656 2672 1408 1424 
	 [42496 42752 22528 22784]
2656 2672 1424 1440 
	 [42496 42752 22784 23040]
2656 2672 1440 1456 
	 [42496 42752 23040 23296]
2656 2672 1456 1472 
	 [42496 42752 23296 23552]
2656 2672 1472 1488 

2704 2720 1664 1680 
	 [43264 43520 26624 26880]
2704 2720 1680 1696 
	 [43264 43520 26880 27136]
2704 2720 1696 1712 
	 [43264 43520 27136 27392]
2704 2720 1712 1728 
	 [43264 43520 27392 27648]
2704 2720 1728 1744 
	 [43264 43520 27648 27904]
2704 2720 1744 1760 
	 [43264 43520 27904 28160]
2704 2720 1760 1776 
	 [43264 43520 28160 28416]
2704 2720 1776 1792 
	 [43264 43520 28416 28672]
2704 2720 1792 1808 
	 [43264 43520 28672 28928]
2704 2720 1808 1824 
	 [43264 43520 28928 29184]
2704 2720 1824 1840 
	 [43264 43520 29184 29440]
2704 2720 1840 1856 
	 [43264 43520 29440 29696]
2704 2720 1856 1872 
	 [43264 43520 29696 29952]
2704 2720 1872 1888 
	 [43264 43520 29952 30208]
2704 2720 1888 1904 
	 [43264 43520 30208 30464]
2704 2720 1904 1920 
	 [43264 43520 30464 30720]
2704 2720 1920 1936 
	 [43264 43520 30720 30976]
2704 2720 1936 1952 
	 [43264 43520 30976 31232]
2704 2720 1952 1968 
	 [43264 43520 31232 31488]
2704 2720 1968 1984 
	 [43264 43520 31488 31744]
2704 2720 1984 2000 

2768 2784 144 160 
	 [44288 44544  2304  2560]
2768 2784 160 176 
	 [44288 44544  2560  2816]
2768 2784 176 192 
	 [44288 44544  2816  3072]
2768 2784 192 208 
	 [44288 44544  3072  3328]
2768 2784 208 224 
	 [44288 44544  3328  3584]
2768 2784 224 240 
	 [44288 44544  3584  3840]
2768 2784 240 256 
	 [44288 44544  3840  4096]
2768 2784 256 272 
	 [44288 44544  4096  4352]
2768 2784 272 288 
	 [44288 44544  4352  4608]
2768 2784 288 304 
	 [44288 44544  4608  4864]
2768 2784 304 320 
	 [44288 44544  4864  5120]
2768 2784 320 336 
	 [44288 44544  5120  5376]
2768 2784 336 352 
	 [44288 44544  5376  5632]
2768 2784 352 368 
	 [44288 44544  5632  5888]
2768 2784 368 384 
	 [44288 44544  5888  6144]
2768 2784 384 400 
	 [44288 44544  6144  6400]
2768 2784 400 416 
	 [44288 44544  6400  6656]
2768 2784 416 432 
	 [44288 44544  6656  6912]
2768 2784 432 448 
	 [44288 44544  6912  7168]
2768 2784 448 464 
	 [44288 44544  7168  7424]
2768 2784 464 480 
	 [44288 44544  7424  7680]
2768 2784 480

2816 2832 656 672 
	 [45056 45312 10496 10752]
2816 2832 672 688 
	 [45056 45312 10752 11008]
2816 2832 688 704 
	 [45056 45312 11008 11264]
2816 2832 704 720 
	 [45056 45312 11264 11520]
2816 2832 720 736 
	 [45056 45312 11520 11776]
2816 2832 736 752 
	 [45056 45312 11776 12032]
2816 2832 752 768 
	 [45056 45312 12032 12288]
2816 2832 768 784 
	 [45056 45312 12288 12544]
2816 2832 784 800 
	 [45056 45312 12544 12800]
2816 2832 800 816 
	 [45056 45312 12800 13056]
2816 2832 816 832 
	 [45056 45312 13056 13312]
2816 2832 832 848 
	 [45056 45312 13312 13568]
2816 2832 848 864 
	 [45056 45312 13568 13824]
2816 2832 864 880 
	 [45056 45312 13824 14080]
2816 2832 880 896 
	 [45056 45312 14080 14336]
2816 2832 896 912 
	 [45056 45312 14336 14592]
2816 2832 912 928 
	 [45056 45312 14592 14848]
2816 2832 928 944 
	 [45056 45312 14848 15104]
2816 2832 944 960 
	 [45056 45312 15104 15360]
2816 2832 960 976 
	 [45056 45312 15360 15616]
2816 2832 976 992 
	 [45056 45312 15616 15872]
2816 2832 992

In [6]:
"""                                                paralell time test
need to know what size the pre-trained models used
& graphic file types preferred by tensorflow (float, float32, int, etc)
"""
PATCH_HEIGHT = 256
PATCH_WIDTH = 256
target_dir = '../../DigiPath_MLTK_data/out_to_test'

t0 = time.time()
# open the file
test_file_name = os.path.join(data_dir, 'CMU-1.svs')
print('Opening Image file: \n\t{}\n'.format(test_file_name))
os_obj = openslide.OpenSlide(test_file_name)
pixels_height = os_obj.dimensions[0]
pixels_width = os_obj.dimensions[1]

print('Image is {} x {}\n{} pixels'.format(pixels_height, 
                                           pixels_width, 
                                           pixels_height * pixels_width))

# read the meta data
pixels_height_ds = os_obj.level_dimensions[-1][0]
pixels_width_ds = os_obj.level_dimensions[-1][1]
print(pixels_height_ds, pixels_width_ds)

# find the thumbnail size ratio (need pre-trained size)
print(pixels_height / pixels_height_ds, pixels_width / pixels_width_ds)
print('obj.level_downsamples', os_obj.level_downsamples)
print('obj.level_downsamples[-1]', os_obj.level_downsamples[-1])

big_thumb = os_obj.get_thumbnail((pixels_height_ds, pixels_width_ds))
print(type(big_thumb), big_thumb.size, big_thumb.getextrema())

# make the mask
grey_thumbnail = np.array(big_thumb.convert('L'))
thresh = threshold_otsu(grey_thumbnail)
mask = np.array(grey_thumbnail) < thresh
mask_im = PIL.Image.fromarray(np.uint8(mask) * 255)
# display(mask_im)

# make the grid
grid_height = pixels_height // PATCH_HEIGHT
grid_width = pixels_width // PATCH_WIDTH
print('\n%40s: %5i, %5i'%('grid size', grid_height, grid_width))
print('%40s: %5i, %5i'%('full size cells',PATCH_HEIGHT, PATCH_WIDTH))
thumb_cell_height = pixels_height_ds // grid_height
thumb_cell_width = pixels_width_ds // grid_width
print('%40s: %5i, %5i'%('thumb cells',thumb_cell_height, thumb_cell_width))

# make the directory
_, f_name = os.path.split(test_file_name)
f_name, f_ext = os.path.splitext(f_name)
print('directory name',f_name)


"""                             N1730 Aug 28 
                            run_grid_row divides rows OK, need TFRecord per row output
                                    AND
                            TFRecord collector afterward
"""
# get the ndarrays of full size grid, write as type
def run_grid_row(run_parameters, grid_row):
    grid_height = run_parameters['grid_height']
    grid_width = run_parameters['grid_width']
    thumb_cell_height = run_parameters['thumb_cell_height']
    thumb_cell_width = run_parameters['thumb_cell_width']
    zoom_up_ratio = run_parameters['zoom_up_ratio']
    target_dir = run_parameters['target_dir']
    
    for grid_col in range(grid_width):
        h_idx_start = grid_row * thumb_cell_height
        h_idx_end = (grid_row + 1) * thumb_cell_height
        w_idx_start = grid_col * thumb_cell_width
        w_idx_end = (grid_col + 1) * thumb_cell_width
        
        big_idx = np.array([np.array([h_idx_start, h_idx_end, w_idx_start, w_idx_end]),
                            np.array([h_idx_start, h_idx_end, w_idx_start, w_idx_end]
                                    ) * zoom_up_ratio])

        s = ''
        for k in big_idx.reshape(-1):
            s += '%i'%k
        f_name = os.path.join(target_dir, hashlib.sha1(s.encode('utf-8')).hexdigest())
        with open(f_name, 'wb') as fh:
            big_idx.dump(fh)
        
zoom_up_ratio = 16
grid_row_enum = np.int_(range(0, grid_height))

temp_dirpath = tempfile.mkdtemp()
run_parameters = {'grid_height': grid_height, 
                  'grid_width': grid_width, 
                  'thumb_cell_height': thumb_cell_height, 
                  'thumb_cell_width': thumb_cell_width, 
                  'zoom_up_ratio': zoom_up_ratio, 
                  'target_dir': temp_dirpath}

n_cores = mp.cpu_count()
grid_pool = mp.Pool(processes=n_cores)
grid_pool.starmap(run_grid_row, 
                  zip(itertools.repeat(run_parameters), grid_row_enum))
grid_pool.close()
grid_pool.join()


"""                             N1730 Aug 28 
                                assemble_tmp_dir takes twice as long as run_grid_row
                                
"""
def assemble_temp_dir(run_parameters):
    target_dir = run_parameters['target_dir']
    index_array_size = grid_height * grid_width
    from_arr = np.zeros((index_array_size, 4))
    too_arr = np.zeros((index_array_size, 4))
    row_counter = 0
    for maybe_file in os.listdir(target_dir):
        maybe_full_file = os.path.join(target_dir, maybe_file)
        if os.path.isfile(maybe_full_file):
            this_mat = np.load(maybe_full_file)
            if isinstance(this_mat, np.ndarray) and this_mat.shape[0] == 2:
                from_arr[row_counter, :] = this_mat[0,:]
                too_arr[row_counter, :] = this_mat[1,:]
                row_counter += 1
                os.remove(maybe_full_file)
    print('\n\tremoved %i files'%(row_counter))
    return from_arr, too_arr

from_arr, too_arr = assemble_temp_dir(run_parameters)

try:
    print('removing', temp_dirpath)
    os.rmdir(temp_dirpath)
except:
    print(temp_dirpath, '\n directory not empty')

print('\nthumb size:', pixels_height_ds, pixels_width_ds)
print('\nfull size:', pixels_height, pixels_width)
print('%i cores available'%(n_cores))
# (save a list of files)

print('\ncell run time', time.time() - t0)

Opening Image file: 
	../../DigiPath_MLTK_data/Aperio/CMU-1.svs

Image is 46000 x 32914
1514044000 pixels
2875 2057
16.0 16.000972289742343
obj.level_downsamples (1.0, 4.000121536217793, 16.00048614487117)
obj.level_downsamples[-1] 16.00048614487117
<class 'PIL.Image.Image'> (2875, 2057) ((0, 255), (0, 255), (3, 255))

                               grid size:   179,   128
                         full size cells:   256,   256
                             thumb cells:    16,    16
directory name CMU-1

	removed 22912 files
removing /var/folders/12/rq_vvnr91sj5kh6xj64x3vsr0000gn/T/tmp5d_04y5k

thumb size: 2875 2057

full size: 46000 32914
4 cores available

cell run time 16.41913604736328


In [46]:
from_arr[0,:], too_arr[0,:]

(array([2288., 2304.,  704.,  720.]), array([36608., 36864., 11264., 11520.]))

## old code
```python
MAXPATCHESTOWRITE = 1000
import glob
import math
import os
import numpy as np
import openslide
from skimage.filters import threshold_otsu

def get_thumbnail(img, x_level0, y_level0, patch_size, output_path):
    patch = img.read_region((x_level0, y_level0), 0, (patch_size, patch_size))
    patch = patch.convert('RGB')
    fname = img.properties['aperio.Filename'].replace(' ', '_')
    fname += '_' + str(x_level0)
    fname += '_' + str(y_level0)
    fname += '_' + '0'
    fname += '_' + str(patch_size)
    fname += '.jpg'
    patch_arr = np.array(patch.convert('L'))
    if np.average(patch_arr) < 230:
        global num_patches
        num_patches += 1
        patch.save(os.path.join(output_path, fname))

def process_svs(SVS,
                normalization_factor=1000,
                patch_size=512,
                buffer=10,
                output_path=OUTPUT_PATH):
    img = openslide.OpenSlide(SVS)

    global num_patches
    num_patches = 0

    x_max, y_max = img.dimensions
    thumbnail = img.get_thumbnail((normalization_factor, normalization_factor))

    grey_thumbnail = np.array(thumbnail.convert("L"))
    thresh = threshold_otsu(grey_thumbnail)
    mask = np.array(grey_thumbnail) < thresh

    # how many pixels in the raw image per pixel in mask
    x_num_orgPix_per_thumbPix = math.ceil(x_max / mask.shape[0])
    y_num_orgPix_per_thumbPix = math.ceil(y_max / mask.shape[1])
    # print(x_num_orgPix_per_thumbPix, y_num_orgPix_per_thumbPix)

    # Find out how many pixels in image mask to count as a patch in original
    num_x_mask_pixels_per_rawPatch = math.ceil(patch_size / x_num_orgPix_per_thumbPix)
    num_y_mask_pixels_per_rawPatch = math.ceil(patch_size / y_num_orgPix_per_thumbPix)
    # print(num_x_mask_pixels_per_rawPatch, num_y_mask_pixels_per_rawPatch)

    mask_x, mask_y = mask.shape
    x_mask_prev = 0

    # Iterate through the mask to identify positive pixels
    for x in range(buffer, mask_x - buffer):
        x_mask_window = x + num_x_mask_pixels_per_rawPatch
        if x_mask_window <= x_mask_prev:
            continue
        y_mask_prev = 0
        for y in range(buffer, mask_y - buffer):
            y_mask_window = y + num_y_mask_pixels_per_rawPatch
            # print('Evaluate: {} {} & {}'.format(y, y_mask_window, y_mask_prev))
            if y_mask_window <= y_mask_prev:
                continue
            if y % 100 == 0:
                print('X: {}\tY:{} of {} with total of {} so far'.format(x, y, 
                                                                         mask.shape, 
                                                                         num_patches), end='\r',
                                                                  flush=True)
                
            if np.sum(mask[x:x_mask_window, y:y_mask_window]) > 0:
                # convert mask coordinates to level0 coordinates
                x_level0 = x * x_num_orgPix_per_thumbPix
                y_level0 = y * y_num_orgPix_per_thumbPix
                get_thumbnail(img, x_level0, y_level0, patch_size, output_path)
                
                #                                      <0><0>          Limit number of files
                if num_patches >= MAXPATCHESTOWRITE:
                    print('\n\n\t{} num_patched out\n\n'.format(num_patches))
                    return
                #                                      <0><0>

            # print('yamsk windoe: {}'.format(y_mask_window))
            y_mask_prev = y_mask_window
        x_mask_prev = x_mask_window

    print('Printed {} from {}'.format(num_patches, SVS))

num_patches = 0
"""
# if __name__ == '__main__':
#     for SVS in glob.glob('/data/biliary/svs/negative/*svs'):
#         process_svs(SVS)
"""
print('processing: {}'.format(SVS))
process_svs(SVS)
```